In [ ]:
import json
import pandas as pd
import psycopg2
import csv

In [ ]:
heart = pd.read_json('~/Desktop/py129/self-viz/sample_heart.json')

# Empty lists for heartbeats/min and Fitbit 'Confidence' value
bpm_list = []
conf_list = []

# Populates empty lists with respective Fitbit values
for i in heart['value']:
    bpm_list.append(i['bpm'])
    conf_list.append(i['confidence'])

# Creates new pandas dataframes from populated lists
bpm_df = pd.DataFrame(bpm_list)
conf_df = pd.DataFrame(conf_list)

# Assigns dataframes as new series/columns in original dataframe
heart['BPM'] = bpm_df
heart['Confidence'] = conf_df

# Reads JSON file and renames column for Calories burned per minute
calor = pd.read_json('~/Desktop/py129/self-viz/sample_calories.json')
calor.rename(columns={'value':'Calories'}, inplace=True)

# Exports dataframes as CSV
heart[['dateTime','BPM','Confidence']].to_csv(('~/Desktop/py129/self-viz/sample_heart.csv'))
calor.to_csv(('~/Desktop/py129/self-viz/sample_calories.csv'))


In [ ]:
#heart = pd.read_csv('~/Desktop/py129/self-viz/denver_trip.csv')

In [ ]:
try:
    # Creates connection to new DB, creates if nonexistant
    conn = psycopg2.connect(
        database='heartcaltrip',
        user='steve_wortmann',
        password='paigek#624',
        host='localhost'
    )
    
    # Cursor set for SQL insertion
    cursor = conn.cursor()
    print("Connected...")
    
    # DB tables to house both BPM 'Heart', Calories, and Geographic 'Trip' data are created
    # Primary integer key is set for all tables
    hearttable_sql='''
                    CREATE TABLE IF NOT EXISTS
                        heart(
                            time_id INTEGER PRIMARY KEY,
                            dateTime TIMESTAMP,
                            Bpm INTEGER,
                            Confidence INTEGER
                        );
                '''
    cursor.execute(hearttable_sql)
    conn.commit()
    print("Created HEART Table...")
    
    calorietable_sql='''
                    CREATE TABLE IF NOT EXISTS
                        calorie(
                            time_id INTEGER PRIMARY KEY,
                            dateTime TIMESTAMP,
                            Calories FLOAT
                        );
                '''
    cursor.execute(calorietable_sql)
    conn.commit()
    print("Created CALORIE Table...")

    triptable_sql='''
                    CREATE TABLE IF NOT EXISTS
                        trip(
                            time_id INTEGER PRIMARY KEY,
                            name TEXT,
                            address TEXT,
                            description TEXT,
                            TimeSpan_begin TIMESTAMP,
                            TimeSpan_end TIMESTAMP,
                            longitude FLOAT,
                            latitude FLOAT,
                            LineString_coordinates TEXT
                        );
                '''
    cursor.execute(triptable_sql)
    conn.commit()
    print("Created TRIP Table...")
    
except psycopg2.Error as error:
        print("Modular Error...")
        print(error)

# Database resources are closed at the end of insertion cycles
finally:
    if cursor:
        cursor.close()
        if conn:
            conn.close()

In [ ]:
# Utility establishing database connection,cursor function returning all access objects
def connect_PostgreSQL():
    try:
        dbconn = psycopg2.connect(
            database='heartcaltrip',
            user='steve_wortmann',
            password='paigek#624',
            host='localhost'
        )
        cursor = dbconn.cursor()
        print("Connected to 'heartcaltrip' database")
    except psycopg2.Error as error:
        print("Error opening database")
        dbconn = handle_DB_error(dbconn, cursor)
    return dbconn, cursor

# Utility to close database resources to call after future insertions
def close_DB_resources(dbconn, cursor):
    try:
        if dbconn:
            dbconn.close()
        if cursor:
            cursor.close()
        print("closed resources")
    except psycopg2.Error as err:
        ("Error closing resources")

# Utility to handle database errors
def handle_DB_error(dbconn, cursor):
    if dbconn:
        try:
            dbconn.rollback()
            print("Rolled back transation")
        except psycopg2.Error as error:
            print("Error rolling back transaction")
        finally:
            close_DB_resources(dbconn, cursor)
            dbconn = None 
            return dbconn

In [ ]:
heart_insert = '''INSERT INTO heart (time_id, dateTime, Bpm, Confidence)
                  VALUES (%s,%s,%s,%s);
                '''

calorie_insert = '''INSERT INTO calorie (time_id, dateTime, Calories)
                    VALUES (%s,%s,%s);
                '''

trip_insert = '''INSERT INTO trip (time_id, name, address, description, TimeSpan_begin, TimeSpan_end, longitude, latitude, LineString_coordinates)
                 VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s);
                '''

# CSV file is opened, columns as 'time' are fed to (%s) fields in SQL insertion statement
# 'value_id' is iterated and assigned with column insertions

# BRM 'Heart' table is populated first from CSV...
dbconn, cursor = connect_PostgreSQL()

with open('sample_heart.csv') as file:
    time_stamps = csv.DictReader(file)
    value_id = 0
    for time in time_stamps:
        value_id += 1
        record = (value_id, time['dateTime'], time['BPM'], time['Confidence'])
        print(record)
        cursor.execute(heart_insert, record)
        dbconn.commit()
    print('All file records committed to database: BPM/HeartRate table...')

close_DB_resources(dbconn, cursor)

# Followed by Calories table...
dbconn, cursor = connect_PostgreSQL()

with open('sample_calories.csv') as file:
    time_stamps = csv.DictReader(file)
    value_id = 0
    for time in time_stamps:
        value_id += 1
        record = (value_id, time['dateTime'], time['Calories'])
        #print(record)
        cursor.execute(calorie_insert, record)
        dbconn.commit()
    print('All file records committed to database: Caloric table...')
        
close_DB_resources(dbconn, cursor)

# Followed by Geographic/Location 'Trip' table...
dbconn, cursor = connect_PostgreSQL()

with open('denver_trip.csv') as file:
    time_stamps = csv.DictReader(file)
    value_id = 0
    for time in time_stamps:
        value_id += 1
        try:
            float(time['longitude'])
        except:
            print('invalid entry')
            time['longitude'] = None
            time['latitude'] = None
        
        record = (value_id, time['\ufeffname'], time['address'], time['description'], time['TimeSpan_begin'], time['TimeSpan_end'], time['longitude'], time['latitude'], time['LineString_coordinates'])
        print(record)
        cursor.execute(trip_insert, record)
        dbconn.commit()
    print('All file records committed to database: Geographic/Location table...')

close_DB_resources(dbconn, cursor)

In [ ]:
# Readings within 'dateTime' are truncated down to their minute value
# Average values for BPM and Confidence readings are taken per minute period
# Summation of Calorie values are taken over for every minute period
# All values are singularly grouped for every minute, then ordered chronologically...

# These values are populated as single postgres View resource...
create_calorie_heartrate_view = '''CREATE VIEW calorie_heartrate AS
                 SELECT date_trunc('minute', h.dateTime) AS dateTime_min,
                 avg(h.Bpm) AS BPM_min_avg, avg(h.Confidence) AS Conf_min_avg,
                 avg(c.calories) AS calories
                 FROM heart AS h
                 JOIN calorie as c
                 ON c.dateTime = date_trunc('minute', h.dateTime)
                 GROUP BY date_trunc('minute', h.dateTime)
                 ORDER BY date_trunc('minute', h.dateTime);
              ''' 

# All newly-generated minute values for Calories, BPM, Confidence are collected from View...
pull_calorie_heartrate = '''SELECT dateTime_min, calories, BPM_min_avg, Conf_min_avg FROM calorie_heartrate;''' 

# First insertion is executed, compended postgres View is created of most-relevant data
# View is then fetched, that data is then stored in a single tuple...

dbconn, cursor = connect_PostgreSQL()

cursor.execute(create_calorie_heartrate_view)
print('Grouping data readings per minute...')
cursor.execute(pull_calorie_heartrate)
tupples = cursor.fetchall()
print('Extracting data approximations from Postgres...')

close_DB_resources(dbconn, cursor)

# That tuple is used to create new pd Dataframe of Calories + Average HeartRate per minute...
calorie_heartrate_df = pd.DataFrame(tupples, columns=['dateTime','calories','bpm_min','conf_min'])
calorie_heartrate_df

In [ ]:
calorie_heartrate_df

In [ ]:
dbconn, cursor = connect_PostgreSQL()

trip_add_columns =  '''ALTER TABLE trip
                       ADD COLUMN calories DOUBLE PRECISION, ADD COLUMN avg_hr NUMERIC, ADD COLUMN avg_conf NUMERIC;
                    '''

cursor.execute(trip_add_columns)
dbconn.commit()
close_DB_resources(dbconn, cursor)

In [ ]:
dbconn, cursor = connect_PostgreSQL()

trip_pull = '''SELECT * FROM trip
               ORDER BY time_id;'''

heart_cal_pull = '''SELECT sum(c_hr.calories) AS calories,
                           avg(c_hr.bpm_min_avg) AS avg_bpm,
                           avg(c_hr.conf_min_avg) AS confidence
                    FROM calorie_heartrate AS c_hr
                    WHERE c_hr.dateTime_min
                    BETWEEN (%s) AND (%s);
                 '''

trip_fill_columns =  '''UPDATE public.trip
                        SET calories = CAST(%s AS DOUBLE PRECISION),
                            avg_hr = (%s),
                            avg_conf = (%s)
                        WHERE
                            time_id = (%s);
                     '''

insertion_record = '''SELECT calories, avg_hr, avg_conf FROM trip
                      WHERE time_id = (%s);
                      '''

final_trip_pull = '''SELECT time_id, name, address, timespan_begin, timespan_end,
                     calories, avg_hr, avg_conf, longitude, latitude, linestring_coordinates, description
                     FROM trip ORDER BY time_id;
                     '''

# Location 'Trip' table is scanned, iterated by its consecutive time periods at a location (or in transit)

# For each arbitrary time period expressed:
#    - Our Postgres View of minutely-segregated calories burned, and average BPM readings, are collected.
#        - Each per-minute reading is then again either summated or averaged to a single value.
#        - These values are used for insertion into their respective column on 'Trip' table, via 'heart_cal_period'.
#        - 'heart_cal_period' matches values within their appropriate time period.

cursor.execute(trip_pull)
trip_record = cursor.fetchall()

print("Total rows are: ", len(trip_record))

for row in trip_record:
    cursor.execute(heart_cal_pull, (row[4], row[5]))
    heart_cal_period = cursor.fetchone()
    if heart_cal_period:
        cursor.execute(trip_fill_columns, (heart_cal_period[0], heart_cal_period[1], heart_cal_period[2], row[0]))
        cursor.execute(insertion_record, (row[0],))
        print(cursor.fetchone())

print("Columns populated: 'calories', 'average_hr', 'avg_conf'")

dbconn.commit()

cursor.execute(final_trip_pull)
trip_tupple = cursor.fetchall()

close_DB_resources(dbconn, cursor)